<a href="https://colab.research.google.com/github/tomormerod/GEOG3003M/blob/main/GEOG5003M_Report_Assignment_200919142.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The relationship between the number of affordable homes and residential sales per Yorkshire and Humber district

#1.0 Package installation
Per coding conventions, all packages and modules therein required for the code to run were downloaded first. Packages not built into Python were installed via the 'pip install' command. One module was imported from Matplotlib.

In [ ]:
# Install neccessary packages
!pip install mapclassify
!pip install contextily
!pip install geoplot
!pip install git+https://github.com/pmdscully/geo_northarrow.git
!pip install -U notebook-as-pdf

# Import neccessary packages
import pyproj
import contextily as ctx
import geoplot as gplt
import geoplot.crs as gcrs

import geopandas as gpd
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

# Import neccessary modules.

from matplotlib.colors import ListedColormap

# Set seaborn plotting theme to white
sns.set_theme(style="white")

#2.0 Non-spatial data preprocessing
## 2.1 Data import
To create this CSV file, the comments, navigation links and overall page header from one sheet of the original XLSX file were removed to make the column headers the first row.

In [ ]:
# Read in and undertake a basic exploration of the Statistical Data Return (SDR) dataset.
sdr = pd.read_csv('https://github.com/tomormerod/GEOG3003M/raw/refs/heads/main/sdr_2024_data.csv')
sdr.head()

In [ ]:
# View the column names. Note that these are not acceptable for further use currently and need cleaning.
sdr.columns

## 2.2 Data cleaning
To clean the column names, they were made lowercase and spaces were removed.

In [ ]:
# Replace all spaces with dashes and make all headers lowercase.
sdr.columns = sdr.columns.str.replace(' ', '_')
sdr.columns = sdr.columns.str.lower()
# View the new column names to refer to in later stages.
sdr.columns

The relevant columns for this analysis, i.e. the number of properties owned by housing associations, were selected.

In [ ]:
# Copy and paste from the list above to select the subset of columns which are relevant to this project.
sdr_red = sdr[['local_authority_name', 'la_code', 'general_needs_-_self-contained_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'general_needs_-_bedspace_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'supported_housing_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'housing_for_older_people_-_owned_low_cost_rental_accommodation_by_local_authority_area']]
# Check for missing data and current data types.
sdr_red.info()

The dataset was also filtered for the Yorkshire & Humber region.

In [ ]:
# Filter the dataset for the region of interest.
sdr_yorks = sdr_red.loc[sdr_red['local_authority_name'].isin(['York', 'Leeds', 'Wakefield', 'Kirklees', 'Bradford', 'Calderdale', 'Barnsley', 'East Riding of Yorkshire',
'Doncaster', 'Kingston upon Hull, City of', 'North East Lincolnshire', 'North Lincolnshire', 'North Yorkshire', 'Rotherham', 'Sheffield'])]

## 2.3 Data aggregation
The data raw consisted of string literals of the numbers, so to be processed, it was transformed to a numerical format. To transform the null values into a number, their representation was changed from ‘[x]’ to ‘0’.

In [ ]:
# Transform 'no data' points to the string literal '0' and transform all the numerical columns from string to numeric.
sdr_yorks = sdr_wyorks.replace('[x]','0')
sdr_yorks[['general_needs_-_self-contained_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'general_needs_-_bedspace_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'supported_housing_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'housing_for_older_people_-_owned_low_cost_rental_accommodation_by_local_authority_area']] = sdr_wyorks[['general_needs_-_self-contained_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'general_needs_-_bedspace_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'supported_housing_-_owned_low_cost_rental_accommodation_by_local_authority_area',
       'housing_for_older_people_-_owned_low_cost_rental_accommodation_by_local_authority_area']].apply(pd.to_numeric)
# View the result.
sdr_yorks.info()

These numerical columns were then grouped to give the subtotal of each property type for each council.

In [ ]:
# Match the 'local authority name' column header to the 'local authority code' column header.
sdr_wyorks = sdr_wyorks.rename(columns={'local_authority_name':'la_name'})
# Grouping by both, sum the various types of property owned by each housing association.
sdr_la = sdr_wyorks.groupby(['la_name', 'la_code']).agg({'general_needs_-_self-contained_-_owned_low_cost_rental_accommodation_by_local_authority_area':'sum',
       'general_needs_-_bedspace_-_owned_low_cost_rental_accommodation_by_local_authority_area':'sum',
       'supported_housing_-_owned_low_cost_rental_accommodation_by_local_authority_area':'sum',
       'housing_for_older_people_-_owned_low_cost_rental_accommodation_by_local_authority_area':'sum'})
# View the result.
sdr_la

The total owned properties across all types for each council was calculated. The columns were renamed for easier recall in a summation formula.

In [ ]:
# Rename the columns for the property types to make them easier to reference.
sdr_la = sdr_la.rename(columns={'general_needs_-_self-contained_-_owned_low_cost_rental_accommodation_by_local_authority_area' : 'gen_sc_owned',
       'general_needs_-_bedspace_-_owned_low_cost_rental_accommodation_by_local_authority_area' : 'gen_bed_owned',
       'supported_housing_-_owned_low_cost_rental_accommodation_by_local_authority_area' : 'sup_owned',
       'housing_for_older_people_-_owned_low_cost_rental_accommodation_by_local_authority_area' : 'elderly_owned'})
# Use these new column names to create a 'total owned' column for each local authority.
sdr_la['total_owned']=sdr_la['gen_sc_owned']+sdr_la['gen_bed_owned']+sdr_la['sup_owned']+sdr_la['elderly_owned']
# View the result.
sdr_la

## 2.4 Further datasets
To explore the relationships between the affordable home ownership and other data, additional datasets were added to and merged with the existing dataset.

### 2.4.1 Residential sales
The original comments and page header from the original CSV file were removed - to make the column headers the first row.
The data was already aggregated to the council level, so it was merged with the existing dataset on the council code. The columns were renamed for clarity.  


In [ ]:
# Read in the 'Residential Sales' dataset.
resi = pd.read_csv('https://github.com/tomormerod/GEOG3003M/raw/refs/heads/main/residential_sales.csv')

# Merge with the SDR dataset to bring the data for each local authority together.
sdr_sales = sdr_la.merge(resi, left_on='la_code', right_on='mnemonic', how='right')
# Set the index to the local authority name.
sdr_sales.set_index('local authority: district / unitary (as of April 2023)')
# Rename the columns from the 'residential sales' part of the dataset to simpler headers.
sdr_sales = sdr_sales.rename(columns = {'local authority: district / unitary (as of April 2023)':'la_name', 'mnemonic':'la_code', 'Sep-24': 'residential_sales'})

### 2.4.2 Council GDP
The statistical region of each council is given. The Yorkshire & Humber ITL1 region was used to filter the data, so they could be merged. Columns were renamed for clarity and consistency.

The merged, cleaned dataset was then reordered according to the size of the GDP, as required for visualisation.


In [ ]:
# Read in the 'GDP LA' dataset.
gdp_la = pd.read_csv('https://github.com/tomormerod/GEOG3003M/raw/refs/heads/main/gdp_la.csv')
# Filter for the Yorkshire & Humber region (the area of interest). Reset the index and then drop the new index column for simplicity.
gdp_la = gdp_la.loc[gdp_la['ITL1 Region']=='Yorkshire and The Humber'].reset_index()
gdp_la = gdp_la.drop(columns = 'index')

# Extract the important columns from the dataset.
gdp_la = gdp_la[['LA code', 'LA name', '2023']]
# Rename the local authority identifier columns to the same as the other datasets and the '2023' column to 'GDP 2023' for clarity.
gdp_la = gdp_la.rename(columns = {'LA code':'la_code', 'LA name':'la_name', '2023':'gdp_2023'})

# Merge the dataset gathered so far.
sdr_sales_gdp = gdp_la.merge(sdr_sales, left_on='la_code', right_on='la_code', how='right')
# There will be two 'la_name' columns, so rename the first one to just 'la_name' and drop the other. Sort values according to GDP, reset the index and drop the new index column.
sdr_sales_gdp = sdr_sales_gdp.rename(columns = {'la_name_x':'la_name'}).sort_values('gdp_2023').reset_index().drop(columns=['index','la_name_y'])

As with the SDR dataset, the columns were transformed from string literals into numerical data. All the commas were first removed as these cannot be interpreted during the transformation. Once converted, the column was further transformed into float data for plotting.  

In [ ]:
# To transform the GDP column from string into numbers, first remove all commas.
sdr_sales_gdp['gdp_2023']= [i.replace(',','') for i in sdr_sales_gdp['gdp_2023']]
# Transform the GDP column into numbers.
sdr_sales_gdp['gdp_2023'] = sdr_sales_gdp['gdp_2023'].apply(pd.to_numeric)
# The column used to define size order will need float data, so create a new column by transforming the GDP, now in integer form, into float and rename for clarity.
sdr_sales_gdp['size_order'] = sdr_sales_gdp['gdp_2023'].astype(float)

## 3.0 Non-spatial data visualisation
This visualisation examined the relationship between the number of affordable homes owned by housing associations and the number of sales in different councils.

The third variable displayed is GDP. Varying circles by size is the most effective way of expressing this. Area is not a very effective visual channel overall (Munzner, 2014). However, of the visual channels considered more effective by Munzner's ranking, position is already accounted for, and angle is not applicable, so area was the most effective visual channel available.

A regression line illustrates the relationship further. To simply the process of showing both variably sized points and a regression line, two subplots were made.

For the scatterplot, the maximum size of 1500 was the largest circle that would not breach the subplot’s boundaries. The ratio of 175:1500 is the same as between the size of the smallest and largest economies.

For the regression plot, the councils were all labelled but to avoid overcrowding in the densest area of the graph, four were selected to be placed differently relative to their point. Several labels were also right-justified rather than left-justified to stay within the subplot boundaries. ‘East Riding of Yorkshire’ was shortened for labelling.

The blue of the circles and the red of the line were chosen for their colourblind accessibility using the tool on this webpage: https://davidmathlogic.com/colorblind/#%23648FFF-%23785EF0-%23DC267F-%23FE6100-%23FFB000, which simulates colour-blindness.


In [ ]:
# This is a minor alteration for the purposes of plotting this long name on the graph.
sdr_sales_gdp = sdr_sales_gdp.replace('East Riding of Yorkshire','East Yorkshire')

# Define the figure as having two subplots.
fig, ax = plt.subplots(1,2, figsize=(16,8), sharex=False, sharey=False)

# For subplot 1, first define the data (Total owned vs residential sales), specifying the size of the circles will be determined by the 'size order' column.
sns.scatterplot(ax=ax[0], x='total_owned', y='residential_sales', data=sdr_sales_gdp, size = 'size_order', sizes=(175,1500), legend = 'brief', color = '#648FFF')

# Get and create a title for a legend for the first subplot.
leg = ax[0].get_legend()
leg.set_title('GDP (millions): 2023')
leg.get_frame().set_alpha(0)

# Set axis and chart titles for the first subplot.
ax[0].set(xlabel="Total Affordable Properties Owned",
       ylabel="Residential Sales",
      title ='Effect of GDP on the relationship between \nHA owned properties and residential sales');

# For the second subplot, first define the data (Total owned vs residential sales), specifying the regression line will be order 2 with no confidence intervals.
sns.regplot(ax=ax[1], x='total_owned', y='residential_sales', data=sdr_sales_gdp, order=2, marker='o', color = '#648FFF', ci=None, line_kws={'color':'Red','linewidth':2})

# Set axis and chart titles for the second subplot.
ax[1].set(xlabel="Total Affordable Properties Owned",
       ylabel="Residential Sales",
      title ='Regression line of the relationship between \nHA owned properties and residential sales');

# These label groups are derived from trial and error. The busiest part of the graph is divided into two groups, with different locations relative to their points to avoid overcrowding.
for line in range(0,sdr_sales_gdp.shape[0]):

  if sdr_sales_gdp['la_name'][line] in ['Kingston upon Hull, City of','North East Lincolnshire','Rotherham','York']:
     plt.text(
          sdr_sales_gdp["total_owned"][line]+350,
          sdr_sales_gdp["residential_sales"][line]-75,
          sdr_sales_gdp["la_name"][line],
          ha='left',
          weight='bold'
     )

# These three local authorities are also in a unique position on the graph which warrants a different position to the rest of the points.
  elif sdr_sales_gdp['la_name'][line] in ['Wakefield','Bradford','Sheffield']:
    plt.text(
          sdr_sales_gdp["total_owned"][line]-5300,
          sdr_sales_gdp["residential_sales"][line]+75,
          sdr_sales_gdp["la_name"][line],
          ha='left',
          weight='bold'
    )

  else:
     plt.text(
          sdr_sales_gdp["total_owned"][line]-300,
          sdr_sales_gdp["residential_sales"][line]+75,
          sdr_sales_gdp["la_name"][line],
          ha='left',
          weight='bold'
     )

# Add a brief note explaining that East Yorkshire is officially 'East Riding of Yorkshire'.
plt.text(
      4000,
      7800,
      '"East Yorkshire" full name : \nEast Riding of Yorkshire',
      fontsize = 10)

# 4.0 Spatial data preprocessing
## 4.1 Data import
The code block below was used once to create the shapefile before it was uploaded to GitHub.

The original file contained the boundaries of all council districts in the UK. It was filtered for the Yorkshire and Humber region.


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#yorks_la = gpd.read_file('/content/drive/MyDrive/la23_boundaries.geojson')
#yorks_la = yorks_la.loc[yorks_la['LAD23NM'].isin(['York', 'Leeds', 'Wakefield', 'Kirklees', 'Bradford', 'Calderdale', 'Barnsley', 'East Riding of Yorkshire',
#'Doncaster', 'Kingston upon Hull, City of', 'North East Lincolnshire', 'North Lincolnshire', 'North Yorkshire', 'Rotherham', 'Sheffield'])]
#yorks_la.to_file('Yorks.geojson')

## 4.2 Custom-made colourmap definition
The same tool as before was used to create a sequence of colours which progressed logically for ordinary sighted viewers and were distinguishable by various colourblind viewers. A colourmap from lime green to reddish pink was chosen.

In [ ]:
# This is a list of colours defined by HEX value, specially chosen to be accessible to colourblind viewers.
my_cmap=ListedColormap(['#9BE494', '#648FFF', '#7D145E', '#9E0000', '#FF0C6C'])

## 4.3 Data categorisation
It was not practical to create a continuous, colourblind accessible colour ramp. To account for the limited number of the colours, the continuous data was transformed into categories based on their quintile.

Two new dataframes, one for each variable, were created with only the councils’ names and their quintile ranking for residential sales and total owned homes.


In [ ]:
# Show the local authorities reordered by their 'total owned' value.
sdr_sales_gdp[['total_owned','la_name']].sort_values('total_owned',ascending=False)

In [ ]:
# Create a new dataframe with only the local authority names and their 'total owned' ranking.

# Reorder the data as shown above but reordering the entire dataset and creating a new one.
owned_sorted = sdr_sales_gdp.sort_values('total owned', ascending=False)
#The local authority name column of that dataset, now in the correct order to easily categorise them, becomes a new independent list.
la_name = owned_sorted['la_name']
# The second list consists of 15 values - one for each local authority - stating which quintile it ranks in. With five colours to accomodate the entire dataset, they are divided into five groups of three.
owned_order = ['1. most owned','1. most owned','1. most owned','2. many owned','2. many owned','2. many owned','3. average owned','3. average owned','3. average owned','4. few owned','4. few owned','4. few owned','5. least owned','5. least owned','5. least owned']
# Create a new dataframe with these columns. Rename the 'name column' to the usual for ease of merging in later blocks.
df1 = pd.DataFrame({'Local Authority': la_name, 'Owned ranking': owned_order})

In [ ]:
# Repeat the process for the second variable. Begin by, once again, showing the local authorites reordered but now by their 'residential sales' value.
sdr_sales_gdp[['residential_sales','la_name']].sort_values('residential_sales',ascending=False)

In [ ]:
# Repeat the database formation process as before, but with the 'residential sales' rankings.

# Reorder.
resi_sorted = sdr_sales_gdp.sort_values('residential_sales', ascending=False)
# Reordered local authority name list.
la_name_2 = resi_sorted['la_name']
# Residential sales ranking list in the same order.
resi_order = ['1. most sales', '1. most sales', '1. most sales', '2. many sales','2. many sales', '2. many sales','3. average sales','3. average sales','3. average sales','4. few sales','4. few sales','4. few sales','5. least sales','5. least sales','5. least sales']
# Create a new dataframe and rename columns.
df2 = pd.DataFrame({'Local Authority': la_name_2, 'Sales ranking': resi_order})

## 4.4 Joining with non-spatial data

The reduced shapefile was read in, merged with the newly created dataframe on the council’s name, and tidied by dropping repetitive columns.


In [ ]:
# Merge the two dataframes into one.
df = df1.merge(df2, left_on='Local Authority', right_on = 'Local Authority', how = 'inner')

# Read in the local authority shapefile.
yorks_la = gpd.read_file('https://github.com/tomormerod/GEOG3003M/raw/refs/heads/main/Yorks.geojson')
# Merge with the dataframe created above to add geometry data to the rankings.
sdr_shp = yorks_la.merge(df, left_on='LAD23NM', right_on='Local Authority', how = 'left')
# Drop the unneccessary 'Local Authority' column.
sdr_shp = sdr_shp.drop(columns='Local Authority')
# View the result.
sdr_shp

# 5.0 Spatial data visualisation

The data is represented spatially as choropleth maps. Each colour represents an equal frequency of observations in different sized bins. Green represents high sales and many affordable homes, and pink low sales and few affordable homes.
The geodataframe was transformed to Web Mercator projection to add a basemap, and the alpha was set to 0.5 to view it. Zoom level 9 highlighted key places without overcrowding the figure. A linewidth of 1 made the council boundaries clear.

The target audience of these figures are housing developer stakeholders, so the map was designed with an assumption of prior knowledge about the economic data and geography, such that these can be read intuitively.


In [ ]:
# To allow the data to be plotted against a basemap, transform to Web Mercator projection (EPSG 3857).
sdr_shp_WM = sdr_shp.to_crs(epsg=3857)

# Define the figure as having two subplots.
fig, ax = plt.subplots(1, 2, figsize=(20, 20))

# Specify the columns for each subplot, the 'owned ranking' for the first and the 'sales ranking' for the second, coloured by the custom-made colourmap.
sdr_shp_WM.plot(ax=ax[0], column='Owned ranking', linewidth = 1, alpha = 0.5, legend = True, cmap = my_cmap)
sdr_shp_WM.plot(ax=ax[1], column='Sales ranking', linewidth = 1, alpha = 0.5, legend = True, cmap = my_cmap)

# Get and set titles for the legends for both subplots.
leg1=ax[0].get_legend()
leg1.set_title('Owned homes - ranking')
leg2=ax[1].get_legend()
leg2.set_title('Sales - ranking')

# Remove the axes from both subplots.
ax[0].set_axis_off()
ax[1].set_axis_off()

# Set titles for what each subplot is showing.
ax[0].set_title('Total Council-Owned Affordable Homes in 2023 (Local Authority)')
ax[1].set_title('Residential Sales in 2023 (Local Authority)')

# Define the basemap for the two subplots, specifying for its CRS to be set to that of the geodataframe.
ctx.add_basemap(ax[0],source=ctx.providers.CartoDB.Voyager, zoom=9, crs=sdr_shp_WM.crs)
ctx.add_basemap(ax[1],source=ctx.providers.CartoDB.Voyager, zoom=9, crs=sdr_shp_WM.crs)

# 6.0 Interpretation
There is a parabolic relationship between affordable housing and residential sales. Councils with both many and few affordable homes had low sales, while those with the highest sales had only a modest number. In historic industrialised areas (e.g. Leeds), it was necessary to attract many workers to live there, and higher wages combine with a large buyers’ market. In smaller, wealthy, fast-growing economies (e.g. York, which also benefits from preserving its antiquity), have not needed many affordable homes. This raises prices and reduces affordability and sales. In poorer areas (e.g. Bradford), there is a need for affordable housing, but it is less profitable for housing associations. There is a taboo around new residents moving to previously impoverished city centres, particularly with wealthier suburbs. However, it is improving the living conditions for these areas’ existing residents, with a large supply of affordable homes and a local need for them, that the public benefit can be most.


# 7.0 References

Jones Lang LaSalle IP, Inc. 2024. JLL New Housing Targets Analysis – Targets by numbers - August 2024. Available at: https://residential.jll.co.uk/insights/research/jll-new-housing-targets-analysis-targets-by-numbers-august-2024 . Accessed 04/07/2025.

Ministry of Housing, Communities and Local Government. 2025. Live tables on dwelling stock (Including vacants). Available at: https://www.gov.uk/government/statistical-data-sets/live-tables-on-dwelling-stock-including-vacants . Accessed 04/07/2025.

Munzner, T. 2014. Visualization Analysis and Design. AK Peters Visualization Series. Boca Raton, FL: CRC Press.
Office for National Statistics. 2024a. Housing Affordability in England and Wales. Available at: https://www.ons.gov.uk/peoplepopulationandcommunity/housing/bulletins/housingaffordabilityinenglandandwales/2024/relateddata . Accessed 26/06/2025.

Office for National Statistics. 2025. Regional gross domestic product: local authorities. Available at: https://www.ons.gov.uk/economy/grossdomesticproductgdp/datasets/regionalgrossdomesticproductlocalauthorities . Accessed 26/06/2025.

Office for National Statistics. 2024b. Local Authority Districts (December 2023) Boundaries UK BFC. Available at: https://geoportal.statistics.gov.uk/datasets/ons::local-authority-districts-december-2023-boundaries-uk-bfc-2/about . Accessed 27/6/2025.

Regulator of Social Housing. 2024. Private registered provider social housing stock and rents in England 2023 to 2024. Available at: https://www.gov.uk/government/statistics/private-registered-provider-social-housing-stock-and-rents-in-england-2023-to-2024 . Accessed 26/06/2025.
